<a href="https://colab.research.google.com/github/filipelyrio/Data_Science/blob/main/Projeto_IBOVESPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo IBOVESPA
---
<center><img alt="B3" width="50%" src="https://exame.com/wp-content/uploads/2020/11/DSC2686bx.jpg?quality=70&strip=info&w=1024"></center>

A **B3 (Brasil, Bolsa, Balcão)** – antiga Bovespa – é a Bolsa de Valores de São Paulo. É o principal mercado de ações do Brasil. É essa instituição que divulga diariamente o **Índice Ibovespa**.[1]
 
O Índice Bovespa (**Ibovespa**) é o mais importante indicador do desempenho médio das cotações das ações negociadas na **B3**. É formado pelas ações com maior volume negociado nos últimos meses. O valor atual representa a quantia, em moeda corrente, de uma carteira teórica de ações, constituída em 2 de janeiro de 1968, a partir de uma aplicação hipotética.[2] Esse indicador representa então a média de desempenho das principais ações negociadas na bolsa.[1] 

Reavaliado a cada quatro meses, o índice é composto pelas ações de companhias listadas na **B3** que correpondem a cerca de 80% do número de negócios e do volume financeiro do nosso mercado de capitais.[3]

Para o cálculo do **Ibovespa**, são utilizados os dados referentes à cotação dos ativos da carteira teórica no dia. No entanto, cada ação possui um peso diferente na fórmula. Portanto, é preciso multiplicar a cotação pelo peso, de forma a encontrar a contribuição do ativo para o índice.[1]

**Objetivos:**

Este estudo visa explorar o histórico dos últimos 10 anos do **Índice Bosvespa**. Os dados serão obtidos diretamento do site **Yahoo Finance**. Alguns gráficos serão plotados para mostrar o comportamento do ativo e para expor as possibilidades do uso da linguagem Python e suas bibliotecas no estudo do Mercado Financeiro. Em seguida continuo em busca de características relevantes da movimentação do índice. Ao final, procuro responder a seguinte indagação: 

**Com base nos dados históricos, é possível afirmar que existe dias da semana mais voláteis e que por isso geram mais oportunidades de negócios?**


---




## 1. Preparação do *notebook*

### 1.1 Importação de bibliotecas

In [ ]:
# Intalação yfinance caso necessário
!pip install yfinance

In [ ]:
!pip install cufflinks

In [3]:
# Importação de módulos e pacotes
import pandas as pd
import yfinance as yf
import plotly
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.express as px
import plotly.graph_objs as go
from scipy import stats
cf.go_offline()
plotly.io.renderers.default = 'colab'

### 1.2 Importação de dados do Yahoo Finance

O código para o Índice Bovespa é **^BVSP**.

Este trabalho foi baseado no últimos 10 anos, portanto, de 17 de fevereiro de 2012 a 17 de fevereiro de 2022. Julguei este período longo o bastante para detectar grandes movimentos e dar consistência aos dados, sem englobar o período de crescimento exponencial dos anos 2000.

In [4]:
# Download da série história do Ibovespa
# Salvar dados na variável ibov
ibov = yf.download("^BVSP", start="2012-02-17", end="2022-02-17" )

[*********************100%***********************]  1 of 1 completed


## 2. Exploração dos Dados

### 2.1 Conhecendo o dataset

In [5]:
# Primeiro registros
ibov.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-02-17,66158.0,66562.0,65823.0,66204.0,66204.0,2352600
2012-02-23,66084.0,66329.0,65590.0,65820.0,65820.0,2322800
2012-02-24,65820.0,66335.0,65820.0,65943.0,65943.0,2119800
2012-02-27,65938.0,65954.0,65068.0,65241.0,65241.0,2226000
2012-02-28,65245.0,66152.0,65240.0,65959.0,65959.0,2717000


In [6]:
# Último registros
ibov.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-10,112462.0,113812.0,112163.0,113359.0,113359.0,13267900
2022-02-11,113368.0,114899.0,113128.0,113495.0,113495.0,0
2022-02-14,113643.0,114167.0,113358.0,113807.0,113807.0,10757600
2022-02-15,113905.0,114819.0,113882.0,114660.0,114660.0,11649600
2022-02-16,114830.0,115734.0,114816.0,115181.0,115181.0,12052100


Observe que na coluna "Volume" aparece um valor "0". Irei investigar se o dataset contém valores nulos e zeros mais à frente.

### Legenda

*   **Date** -> Data do pregão
*   **Open** -> Valor de abertura
*   **High** -> Máxima do dia
*   **Low** -> Mínima do dia
*   **Close** -> Valor de fechamento
*   **Adj Close** -> Valor de fechamento ajustado*
*   **Volume** -> Volume financeiro negociado no pregão

*O preço de fechamento ajustado acontece quando uma ação, dentro de qualquer dia de negociação, tem o seu preço alterado para incluir quaisquer distribuições de proventos e ações corporativas que ocorreram em qualquer momento antes da abertura do dia seguinte[4].





In [7]:
# Verificação de valores faltantes
ibov.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [8]:
# Resumo estatístico
ibov.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2471.000000,2471.000000,2471.000000,2471.000000,2471.000000,2.471000e+03
mean,74101.058681,74824.759611,73389.848644,74119.368677,74119.368677,5.088324e+06
std,24342.853027,24523.667948,24158.887899,24357.256650,24357.256650,3.008000e+06
min,37501.000000,38031.000000,37046.000000,37497.000000,37497.000000,0.000000e+00
25%,53810.000000,54251.500000,53227.500000,53804.000000,53804.000000,3.161650e+06
50%,64251.000000,64835.000000,63720.000000,64250.000000,64250.000000,3.972200e+06
75%,96273.000000,97397.000000,95439.500000,96302.500000,96302.500000,5.865250e+06
max,130776.000000,131190.000000,129526.000000,130776.000000,130776.000000,2.176870e+07


Nota-se que a cotação mínima desta série foi 37046 pontos e a máxima de 131190. Média dos 10 anos em 74059 e desvio padrão de 24318. 

In [9]:
# Verificando dados faltantes
ibov.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [10]:
# Verificando coluna "Volume" com valor "0"
ibov.Volume[ibov.Volume == 0].value_counts()

0    24
Name: Volume, dtype: int64

Foram encontrados 24 registros com Voluma = 0. Não usaremos a coluna "Volume" neste estudo, por isso estes dados serão mantidos.

### 2.2 Traçando gráficos

#### 2.2.1 Gráfico de Linha

In [11]:
# Gráfico de linha
fig = go.Figure([go.Scatter(x=ibov.index, y=ibov['Adj Close'], name='IBOV')])
fig.update_layout(title="Índice BOVESPA")

# Anotação relevante
fig.update_layout(annotations=[{
    'text':'Impacto da Pandemia',
    'xref':'x', 
    'yref':'y', 
    'x':'2020-02-20', 
    'y':118000,
    'showarrow':True,
    'ax': 0,
    'ay': -80,
    'font':dict(
            family="Nunito",
            size=15,
            color="#e67e22"
            )
    },{
    'text':'Topo histórico',
    'xref':'x', 
    'yref':'y', 
    'x':'2021-06-07', 
    'y':133000,
    'showarrow':False,
    'ax': 0,
    'ay': -80,
    'font':dict(
            family="Nunito",
            size=15,
            color="black"
            )
    },{
    'text':'Linha de tendência de alta',
    'xref':'x', 
    'yref':'y', 
    'x':'2019-01-20', 
    'y':59000,
    'showarrow':False,
    'font':dict(
            family="Nunito",
            size=15,
            color="green"
            )
    },{
    'text':'Mínima do período',
    'xref':'x', 
    'yref':'y', 
    'x':'2016-01-26', 
    'y':35000,
    'showarrow':False,
    'font':dict(
            family="Nunito",
            size=15,
            color="black"
            )
    }])

fig.add_trace(go.Scatter(
    x=['2016-01-20', '2020-03-10'],
    y=[37400, 92000],
    mode="lines",
    marker_color="green"
    ))

fig.update_layout(showlegend=False)

fig.show()

Observa-se no gráfico um período de lateralidade do IBOVESPA no período de 2012 a 2016. Apos atingir a menor cotação do período (indicado no gráfico) inicia-se um forte movimento de subida (Linha de Tendência de Alta) interrompido apenas pela pandemia de COVID-19. A partir de meados de 2020 até o presente observa-se um recuperação da Bolsa de valores, inclusive, estabelecendo um topo histórico de 131190 pontos. 




#### 2.2.2 Gráfico Candlesticks

No dia-a-dia do Mercado Financeiro, a forma gráfica mais utilizada são os *candlesticks*.

Candlestick é um termo em inglês que significa candelabro. O nome se deve pois o formato gráfico do preço dos ativos lembra o formato de uma vela ou de um candelabro em algumas situações. Essa técnica surgiu no século XVIII, no Japão.[5]

In [12]:
# Gráfico de Candlesticks
meses = ibov.loc['2021-08-17':'2022-02-17']
fig = go.Figure([go.Candlestick(x=meses.index, 
open=meses['Open'],
high=meses['High'],
low=meses['Low'],
close=meses['Close'])])
fig.update_layout(title="Índice BOVESPA - 6 meses")
fig.update_layout(xaxis_rangeslider_visible=False)


fig.add_trace(go.Scatter(
    x=['2021-08-30', '2022-01-13'],
    y=[121000, 104000],
    mode="lines",
    marker_color="red"
    ))

fig.add_trace(go.Scatter(
    x=['2021-11-15', '2022-01-31'],
    y=[101000, 101000],
    mode="lines",
    marker_color="black"
    ))

fig.update_layout(annotations=[{
    'text':'Linha de tendência de baixa',
    'xref':'x', 
    'yref':'y', 
    'x':'2021-11-11', 
    'y':115000,
    'showarrow':False,
    'font':dict(
            family="Nunito",
            size=15,
            color="red"
            )
    },{
    'text':'Suporte de preço',
    'xref':'x', 
    'yref':'y', 
    'x':'2021-12-23', 
    'y':100000,
    'showarrow':False,
    'font':dict(
            family="Nunito",
            size=15,
            color="black"
            )
    }])

fig.update_layout(showlegend=False)

fig.show()

Para ilustrar o uso deste tipo de gráfico selecionei os últimos 6 meses de pregão. Observa-se uma forte tendência de queda (linha vermelha) interrompida após a cotação explorar por duas vezes a região de preço indicada pela reta (Suporte de preço). Após isso, o Índice rompe a tendência de baixa e inicia um movimento rápido de alta.

Este movimento (de baixa) é apenas uma **tendência secundária** dentro da **tendência primária** de alta mostrada no gráfico anterior.

### 2.3 Estudo volatilidade do mercado

#### 2.3.1 Variação diária

In [13]:
# Cálculo variação diária
ibov['var_diaria'] = (ibov['Adj Close']).pct_change()
var_diaria = ibov.var_diaria.dropna()

In [14]:
var_diaria.describe()*100

count    247000.000000
mean          0.035156
std           1.590334
min         -14.779679
25%          -0.819403
50%           0.029614
75%           0.903955
max          13.908215
Name: var_diaria, dtype: float64

In [15]:
# Histograma
fig = px.histogram(var_diaria)
fig.update_layout(showlegend=False)
fig.show()

Apesar de registrar variações extremas de -14% e +13%, a média diária é de +0.035%, indicando uma baixa volatilidade considerando apenas os valores de fechamento.

In [16]:
# Gráfico de Volatilidade
fig = px.line(
    var_diaria,
    title="Volatilidade IBOVESPA"
)

fig.layout.yaxis.tickformat = ".2%"
fig.update_layout(annotations=[{
    'text':'Impacto da Pandemia',
    'xref':'x', 
    'yref':'y', 
    'x':'2020-03-12', 
    'y': 0.15,
    'showarrow':False,
    'font':dict(
            family="Nunito",
            size=15,
            color="#e67e22"
            )
    }])
fig.update_layout(showlegend=False)

Este gráfico corrobora a informação anterior. Apesar de indicar picos de volatilidade, na grande maioria dos pregões a variação não ultrapassa os 5%.

#### 2.3.2 Variação por dia da semana

In [17]:
# Identificando os dias da semana
var_diaria = ibov.var_diaria
var_diaria.index.dayofweek

Int64Index([4, 3, 4, 0, 1, 2, 3, 4, 0, 1,
            ...
            3, 4, 0, 1, 2, 3, 4, 0, 1, 2],
           dtype='int64', name='Date', length=2471)

In [18]:
# Nomeando dias da semana
var_por_dia_da_semana = pd.DataFrame(index=var_diaria.index)
dias_da_semana = ['segundas', 'terças', 'quartas', 'quintas', 'sextas']

for i, dia in enumerate(dias_da_semana):
  var_por_dia_da_semana[dia] = var_diaria[var_diaria.index.dayofweek == i]

In [19]:
# Verificando
var_por_dia_da_semana

,segundas,terças,quartas,quintas,sextas
Date,,,,,
2012-02-17,NaN,NaN,NaN,NaN,NaN
2012-02-23,NaN,NaN,NaN,-0.005800,NaN
2012-02-24,NaN,NaN,NaN,NaN,0.001869
2012-02-27,-0.010646,NaN,NaN,NaN,NaN
2012-02-28,NaN,0.011005,NaN,NaN,NaN
...,...,...,...,...,...
2022-02-10,NaN,NaN,NaN,0.007985,NaN
2022-02-11,NaN,NaN,NaN,NaN,0.001200
2022-02-14,0.002749,NaN,NaN,NaN,NaN


In [20]:
# Boxplot por dia da semana
fig = px.box(var_por_dia_da_semana,
             labels={'variable':'Dia da semana', 'value':"variação_ibovespa"},
             title='Variação por dia da semana - IBOVESPA',
             )

fig.layout.yaxis.tickformat = ".2%"
fig.update_layout(showlegend = False)

Estes gráfico indicam visualmente um homogeneidade na movimentação do Índice. Nenhum dia parece destacar-se dos demais, porém faremos um teste de hipóteses ao final deste trabalho.

In [21]:
# Calculando média de variação por dia
media = var_por_dia_da_semana.mean()

fig = px.bar(media,
             labels={'variable':'IBOVESPA','value':'variação_ibov', 'index':'Dia da semana'},
             title='Variação média por dia da semana',
             text = media
             )

fig.update_traces(texttemplate='%{text:.2%}')
fig.layout.yaxis.tickformat = ".2%"
fig.update_layout(showlegend = False)

Neste ponto é interesante notar que apesar do **IBOVESPA** apresentar uma alta consistente nos últimos 10 anos, as segundas-feiras e as sextas-feiras apresentam-se como dias com tendência de queda.

In [22]:
var_por_dia_da_semana.describe()

,segundas,terças,quartas,quintas,sextas
count,492.000000,492.000000,504.000000,494.000000,488.000000
mean,-0.000452,0.001181,0.000791,0.000312,-0.000088
std,0.017242,0.015124,0.015265,0.016051,0.015769
min,-0.139215,-0.048657,-0.103488,-0.147797,-0.055089
25%,-0.008734,-0.007345,-0.008030,-0.008347,-0.008420
50%,0.000218,0.000465,0.000703,0.000296,-0.000017
75%,0.008246,0.008910,0.010017,0.009249,0.008334
max,0.065216,0.096885,0.074962,0.065972,0.139082


In [23]:
# Cálculo do percentual de dias com variação positiva
var_percentual = var_por_dia_da_semana.agg(lambda var: (var[var>0].count() / var.count()))

fig = px.bar(var_percentual,
             labels={"variable":"Dia da semana", "value":"% de variação positiva", "index":"Dia da semana"},
             title = "% de variação positiva por Dia da Semana - IBOVESPA",
             text = var_percentual,
             )

fig.update_traces(texttemplate='%{text:.2%}')
fig.layout.yaxis.tickformat = ".2%"
fig.update_layout(showlegend = False)

In [24]:
desvio_padrao = var_por_dia_da_semana.std()

fig = px.bar(desvio_padrao,
             labels={"value":"Volatilidade", "index":"Dia da Semana"},
             title='Volatilidade por dia da semana - IBOVESPA',
             text = desvio_padrao,
             )

fig.update_traces(texttemplate='%{text:.2%}')
fig.layout.yaxis.tickformat = ".2%"
fig.update_layout(showlegend = False)

Até o momento nada parece indicar uma diferença significativa de comportamento do índice. Logo em seguida faremos um teste de hipóteses.

#### 2.3.3 Teste de Hipóteses

Buscando corroborar a afirmação de que não há diferença significativa do padrão de movimentação do **IBOVESPA** farei o seguinte teste de hipótese:

Hipótese **H0 -  NÃO HÁ diferença estatística significativa** 

Hipótese **H1 - HÁ diferença estatística significativa**

In [25]:
# Definindo alpha
alpha = 0.05

In [26]:
# Cálculo de p_valores
p_valores = pd.DataFrame(index=dias_da_semana, columns=dias_da_semana)

for dia1 in dias_da_semana:
  for dia2 in dias_da_semana:
    p_valores.at[dia1, dia2] = stats.ttest_ind(var_por_dia_da_semana[dia1].dropna(),
                                               var_por_dia_da_semana[dia2].dropna())[1]

In [27]:
p_valores

,segundas,terças,quartas,quintas,sextas
segundas,1.0,0.114431,0.228239,0.471274,0.73027
terças,0.114431,1.0,0.685296,0.381782,0.198617
quartas,0.228239,0.685296,1.0,0.629255,0.372434
quintas,0.471274,0.381782,0.629255,1.0,0.693416
sextas,0.73027,0.198617,0.372434,0.693416,1.0


In [28]:
# Verificação 
p_valores < alpha

,segundas,terças,quartas,quintas,sextas
segundas,False,False,False,False,False
terças,False,False,False,False,False
quartas,False,False,False,False,False
quintas,False,False,False,False,False
sextas,False,False,False,False,False


In [29]:
# Resultado do Teste de Hipóteses para cada dia da semana
p_valores.applymap(lambda pvalor: "H1" if pvalor < alpha else "H0")

,segundas,terças,quartas,quintas,sextas
segundas,H0,H0,H0,H0,H0
terças,H0,H0,H0,H0,H0
quartas,H0,H0,H0,H0,H0
quintas,H0,H0,H0,H0,H0
sextas,H0,H0,H0,H0,H0


## 3. Conclusão

Respondendo a questão da introdução deste trabalho: 

"Com base nos dados históricos, é possível afirmar que existe dias da semana mais voláteis e que por isso geram mais oportunidades de negócios?"

Afirmo que **não é possível** indicar qual dia ou dias são mais propícios para operações na Bolsa de Valores baseado na volatilidade. Como explicitado pelo Teste de Hipóteses, não há diferença estatística significante que corrobore esta espectativa. 

Este foi um trabalho simples e inicial que poderá servir como base para pesquisas futuras envolvendo estudo de correlação entre diversos ativos e até mesmo implementação de modelos de regressão.

Dúvidas, sugestões e correções são sempre bem vindas.

### Referências

    [1] https://warren.com.br/magazine/indice-ibovespa. Acessado em 17/02/2022.
    [2] https://pt.wikipedia.org/wiki/Ibovespa. Acessado em 17/02/2022.
    [3] https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm. Acessado em 17/02/2022.
    [4] https://www.suno.com.br/artigos/preco-fechamento-ajustado. Acessado em 17/02/2022.
    [5] https://master.clear.com.br/candlestick. Acessado em 17/02/2022.